## K-FOLD CROSS VALIDATION OVER THE PHQ-8 DL MODEL

### Import Libraries

In [8]:
from sklearn.model_selection import KFold
from data_Loader import dataCreator, dataLoaderPickle, justDatasetLoaderPickle
from networks import FeaturesModel
from networks_v2 import AttentionModel
from optimizer import train2, test2, optimizerNet
from optimizer import IterMeter

import os
import torch
import torch.nn as nn
from comet_ml import Experiment

import matplotlib.pyplot as plt 
import numpy as np

import torch.nn.functional as F 
from sklearn.metrics import confusion_matrix
import seaborn as sn 
import pandas as pd 

from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset


### Loading dataset

In [9]:
dataset = justDatasetLoaderPickle('BALANCED_DATASET_ENG_SCRIPTED_FULL')

### Defining train functions

In [10]:
def train2(model, device, train_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment):
    acc_list = []
    loss_list = []
    labels_list = []
    predicted_list = []
    loss_tot_list = []
    accuracy = []

    for epoch in range(epochs):
        for x, y in train_data:
            optimizer.zero_grad()
            z = model(x.float())

            labels_list.extend(y.detach().numpy())
            z_pred = F.log_softmax(z, dim=1)
            z_pred = (torch.max(torch.exp(z_pred), 1)[1])
            predicted_list.extend(z_pred.detach().numpy())
            y = y.long()
            #y = y.float()
            z = z.squeeze(0)
            #print("Label size: ", y.size())
            loss = criterion(z, y)
            loss.backward()
            optimizer.step()
            #scheduler.step()
            iter_meter.step()

            loss_list.append(loss.item())
            #Track accuracy
            total = y.size(0)
            _, predicted = torch.max(z.data, 1)
            correct = (predicted == y).sum().item()
            acc_list.append(correct/total)
            
        print('Train Epoch: {} \tLoss: {:.4f}\tAccuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_list)
        ))
        loss_tot_list.append(np.mean(loss_list))
        accuracy.append(np.mean(acc_list))   
    
    #Printing Confusion matrix
    classes = ('0','1','2','3','4')
    cf_matrix = confusion_matrix(labels_list, predicted_list)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix)*10, index = [i for i in classes], columns=[i for i in classes])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Training')
    
    #Printing training behaviour 
    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Training Loss')
    axs[0].set(xlabel= 'Epoch',ylabel='Loss')
    axs[1].plot(range(epochs), accuracy)
    axs[1].set_title('Training Accuracy')
    axs[1].set(xlabel= 'Epoch',ylabel='Accuracy')
    plt.show()

In [11]:
def test2(model, device, test_data, criterion, optimizer, scheduler, epochs, iter_meter, experiment):   
    acc_list = []
    loss_list = []
    labels_list = []
    predicted_list = []
    loss_tot_list = []
    accuracy = []

    for epoch in range(epochs):
        for x, y in test_data:
            optimizer.zero_grad()
            z = model(x.float())
            labels_list.extend(y.detach().numpy())
            z_pred = F.log_softmax(z, dim=1)
            z_pred = (torch.max(torch.exp(z_pred), 1)[1])
            predicted_list.extend(z_pred.detach().numpy())
            y = y.long()
            #y = y.float()
            z = z.squeeze(0)
            #print("Label size: ", y.size())
            loss = criterion(z, y)

            loss.backward()
            optimizer.step()
            #scheduler.step()
            iter_meter.step()

            loss_list.append(loss.item())
            #Track accuracy
            total = y.size(0)
            _, predicted = torch.max(z.data, 1)
            correct = (predicted == y).sum().item()
            acc_list.append(correct/total)
            
        print('Test Epoch: {} \tLoss: {:.4f}\tAccuracy: {:.4f}'.format(
            epoch,
            np.mean(loss_list),
            np.mean(acc_list)
        ))
        loss_tot_list.append(np.mean(loss_list))
        accuracy.append(np.mean(acc_list))    
    
    classes = ('0','1','2','3','4')
    cf_matrix = confusion_matrix(labels_list, predicted_list)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix)*10, index = [i for i in classes], columns=[i for i in classes])
    plt.figure(figsize=(12,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig('Confusion Matrix Test')

    fig, axs = plt.subplots(2)
    axs[0].plot(range(epochs), loss_tot_list)
    axs[0].set_title('Test Loss')
    axs[0].set(xlabel= 'Epoch',ylabel='Loss')
    axs[1].plot(range(epochs), accuracy)
    axs[1].set_title('Test Accuracy')
    axs[1].set(xlabel= 'Epoch',ylabel='Accuracy')
    plt.show()

### Defining hyperparameters and main

In [12]:
#Constants
learning_Rate = 0.0005
batch_size = 128
epochs = 50
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}
experiment = Experiment(api_key='dummy_key', disabled=True)

hparams = {
    "n_cnn_layers": 6,
    "n_rnn_layers": 1,
    "rnn_dim": 256,
    "h_rnn_layers": 128,
    "n_class": 5,
    "n_feats": 64,
    "stride": 2,
    "dropout": 0.3,
    "learning_rate": learning_Rate,
    "batch_size": batch_size,
    "epochs": epochs
}

experiment.log_parameters(hparams)
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


In [13]:
#Building and training Model
model = AttentionModel(
    hparams['n_cnn_layers'],
    hparams['rnn_dim'],
    hparams['h_rnn_layers'],
    hparams['n_rnn_layers'],
    hparams['n_class'],
    hparams['stride'],
    hparams['dropout']
).to(device).float()

print(model)
print('Number Model Parameters', sum([param.nelement() for param in model.parameters()]))

#Optimizing Model
optimizer, scheduler = optimizerNet(model, hparams)
weights = [0.95, 0.85, 1.0, 0.9, 1.0]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
criterion2 = nn.MSELoss()
iter_meter = IterMeter()

AttentionModel(
  (dense): Conv1d(500, 32, kernel_size=(1,), stride=(1,))
  (cnn): Sequential(
    (0): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (1): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
      )
      (relu): ReLU()
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (2): ResCNN(
      (conv1): Sequential(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runn

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -stdv, stdv)


### K-Fold Cross-Validation

In [14]:
history = {'Train loss':[], 'Test Loss':[], 'Train Accuracy':[], 'Test Accuracy':[]}

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    print('Fold {}'.format(fold+1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

    model = AttentionModel(
        hparams['n_cnn_layers'],
        hparams['rnn_dim'],
        hparams['h_rnn_layers'],
        hparams['n_rnn_layers'],
        hparams['n_class'],
        hparams['stride'],
        hparams['dropout']
    ).to(device).float()
    optimizer, scheduler = optimizerNet(model, hparams)

    train2(model, device, train_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)
    test2(model, device, test_loader, criterion, optimizer, scheduler, epochs, iter_meter, experiment)

    

Fold 1
Train Epoch: 0 	Loss: 1.5810	Accuracy: 0.2621
Train Epoch: 1 	Loss: 1.5660	Accuracy: 0.2750
Train Epoch: 2 	Loss: 1.5577	Accuracy: 0.2804
Train Epoch: 3 	Loss: 1.5523	Accuracy: 0.2847
Train Epoch: 4 	Loss: 1.5485	Accuracy: 0.2877
Train Epoch: 5 	Loss: 1.5454	Accuracy: 0.2905
Train Epoch: 6 	Loss: 1.5430	Accuracy: 0.2927
Train Epoch: 7 	Loss: 1.5411	Accuracy: 0.2943
Train Epoch: 8 	Loss: 1.5393	Accuracy: 0.2960
Train Epoch: 9 	Loss: 1.5375	Accuracy: 0.2981
Train Epoch: 10 	Loss: 1.5357	Accuracy: 0.2998
Train Epoch: 11 	Loss: 1.5344	Accuracy: 0.3014
Train Epoch: 12 	Loss: 1.5330	Accuracy: 0.3029
Train Epoch: 13 	Loss: 1.5316	Accuracy: 0.3043
Train Epoch: 14 	Loss: 1.5301	Accuracy: 0.3059
Train Epoch: 15 	Loss: 1.5286	Accuracy: 0.3075
Train Epoch: 16 	Loss: 1.5271	Accuracy: 0.3091
Train Epoch: 17 	Loss: 1.5257	Accuracy: 0.3105
Train Epoch: 18 	Loss: 1.5245	Accuracy: 0.3117
Train Epoch: 19 	Loss: 1.5232	Accuracy: 0.3132
Train Epoch: 20 	Loss: 1.5219	Accuracy: 0.3144
Train Epoch: 21 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.5931	Accuracy: 0.2917
Train Epoch: 1 	Loss: 1.5694	Accuracy: 0.3095
Train Epoch: 2 	Loss: 1.5529	Accuracy: 0.3156
Train Epoch: 3 	Loss: 1.5372	Accuracy: 0.3169
Train Epoch: 4 	Loss: 1.5245	Accuracy: 0.3193
Train Epoch: 5 	Loss: 1.5141	Accuracy: 0.3211
Train Epoch: 6 	Loss: 1.5056	Accuracy: 0.3229
Train Epoch: 7 	Loss: 1.4987	Accuracy: 0.3248
Train Epoch: 8 	Loss: 1.4929	Accuracy: 0.3266
Train Epoch: 9 	Loss: 1.4877	Accuracy: 0.3282
Train Epoch: 10 	Loss: 1.4833	Accuracy: 0.3297
Train Epoch: 11 	Loss: 1.4792	Accuracy: 0.3304
Train Epoch: 12 	Loss: 1.4739	Accuracy: 0.3312
Train Epoch: 13 	Loss: 1.4687	Accuracy: 0.3320
Train Epoch: 14 	Loss: 1.4638	Accuracy: 0.3332
Train Epoch: 15 	Loss: 1.4592	Accuracy: 0.3348
Train Epoch: 16 	Loss: 1.4549	Accuracy: 0.3356
Train Epoch: 17 	Loss: 1.4510	Accuracy: 0.3367
Train Epoch: 18 	Loss: 1.4474	Accuracy: 0.3375
Train Epoch: 19 	Loss: 1.4440	Accuracy: 0.3388
Train Epoch: 20 	Loss: 1.4409	Accuracy: 0.3398
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.6004	Accuracy: 0.2328
Train Epoch: 1 	Loss: 1.5968	Accuracy: 0.2407
Train Epoch: 2 	Loss: 1.5948	Accuracy: 0.2441
Train Epoch: 3 	Loss: 1.5921	Accuracy: 0.2463
Train Epoch: 4 	Loss: 1.5862	Accuracy: 0.2519
Train Epoch: 5 	Loss: 1.5812	Accuracy: 0.2566
Train Epoch: 6 	Loss: 1.5771	Accuracy: 0.2603
Train Epoch: 7 	Loss: 1.5689	Accuracy: 0.2675
Train Epoch: 8 	Loss: 1.5612	Accuracy: 0.2747
Train Epoch: 9 	Loss: 1.5546	Accuracy: 0.2806
Train Epoch: 10 	Loss: 1.5491	Accuracy: 0.2857
Train Epoch: 11 	Loss: 1.5444	Accuracy: 0.2898
Train Epoch: 12 	Loss: 1.5402	Accuracy: 0.2937
Train Epoch: 13 	Loss: 1.5363	Accuracy: 0.2971
Train Epoch: 14 	Loss: 1.5327	Accuracy: 0.3005
Train Epoch: 15 	Loss: 1.5293	Accuracy: 0.3043
Train Epoch: 16 	Loss: 1.5262	Accuracy: 0.3076
Train Epoch: 17 	Loss: 1.5232	Accuracy: 0.3109
Train Epoch: 18 	Loss: 1.5203	Accuracy: 0.3141
Train Epoch: 19 	Loss: 1.5178	Accuracy: 0.3169
Train Epoch: 20 	Loss: 1.5152	Accuracy: 0.3198
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.6037	Accuracy: 0.2301
Train Epoch: 1 	Loss: 1.5934	Accuracy: 0.2443
Train Epoch: 2 	Loss: 1.5791	Accuracy: 0.2602
Train Epoch: 3 	Loss: 1.5696	Accuracy: 0.2710
Train Epoch: 4 	Loss: 1.5516	Accuracy: 0.2802
Train Epoch: 5 	Loss: 1.5332	Accuracy: 0.2876
Train Epoch: 6 	Loss: 1.5181	Accuracy: 0.2941
Train Epoch: 7 	Loss: 1.5060	Accuracy: 0.2980
Train Epoch: 8 	Loss: 1.4959	Accuracy: 0.3021
Train Epoch: 9 	Loss: 1.4875	Accuracy: 0.3059
Train Epoch: 10 	Loss: 1.4800	Accuracy: 0.3093
Train Epoch: 11 	Loss: 1.4736	Accuracy: 0.3120
Train Epoch: 12 	Loss: 1.4678	Accuracy: 0.3142
Train Epoch: 13 	Loss: 1.4627	Accuracy: 0.3162
Train Epoch: 14 	Loss: 1.4581	Accuracy: 0.3182
Train Epoch: 15 	Loss: 1.4539	Accuracy: 0.3200
Train Epoch: 16 	Loss: 1.4499	Accuracy: 0.3220
Train Epoch: 17 	Loss: 1.4461	Accuracy: 0.3237
Train Epoch: 18 	Loss: 1.4426	Accuracy: 0.3254
Train Epoch: 19 	Loss: 1.4393	Accuracy: 0.3270
Train Epoch: 20 	Loss: 1.4361	Accuracy: 0.3285
Train Epoch: 21 	Loss: 

c:\Users\ericq\OneDrive\Escriptori\TELECOM\MSC MATT\TFM\The research question\The Solution\CNN+RNN\networks_v2.py:67: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -stdv, stdv)


Train Epoch: 0 	Loss: 1.5710	Accuracy: 0.2485
Train Epoch: 1 	Loss: 1.5446	Accuracy: 0.2680
Train Epoch: 2 	Loss: 1.5270	Accuracy: 0.2832
Train Epoch: 3 	Loss: 1.5154	Accuracy: 0.2906
Train Epoch: 4 	Loss: 1.5061	Accuracy: 0.2961
Train Epoch: 5 	Loss: 1.4981	Accuracy: 0.3021
Train Epoch: 6 	Loss: 1.4917	Accuracy: 0.3065
Train Epoch: 7 	Loss: 1.4864	Accuracy: 0.3102
Train Epoch: 8 	Loss: 1.4818	Accuracy: 0.3133
Train Epoch: 9 	Loss: 1.4778	Accuracy: 0.3160
Train Epoch: 10 	Loss: 1.4742	Accuracy: 0.3182
Train Epoch: 11 	Loss: 1.4709	Accuracy: 0.3203
Train Epoch: 12 	Loss: 1.4680	Accuracy: 0.3220
Train Epoch: 13 	Loss: 1.4654	Accuracy: 0.3237
Train Epoch: 14 	Loss: 1.4630	Accuracy: 0.3250
Train Epoch: 15 	Loss: 1.4609	Accuracy: 0.3263
Train Epoch: 16 	Loss: 1.4589	Accuracy: 0.3274
Train Epoch: 17 	Loss: 1.4570	Accuracy: 0.3284
Train Epoch: 18 	Loss: 1.4552	Accuracy: 0.3294
Train Epoch: 19 	Loss: 1.4536	Accuracy: 0.3303
Train Epoch: 20 	Loss: 1.4521	Accuracy: 0.3312
Train Epoch: 21 	Loss: 